<!--
Doc Writer email@nixdabei.de
v0.0.1, 2022-03-11
-->
[Home](../index.ipynb) / MPU-6050 Beschleunigungssensor: 3D-Cube Beispiel
***
# MPU-6050 Beschleunigungssensor: 3D-Cube Beispiel
## Klassen: [Vector3D](../010_EinfuehrungInJupyterUndPython/OperatorOverloading.ipynb#Klasse-Vector3D), [MPU6050](index.ipynb#Klasse-MPU6050), [AverageFilter](../010_EinfuehrungInJupyterUndPython/RingAndAverageBuffer.ipynb#Klasse-AverageFilter)

[<img src="resources/3DCubeDemoVideo.png" width="600">](resources/3DCubeDemoMp4.ipynb)

In [10]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [11]:
########################################################################
# Vector3D
########################################################################
from math import sqrt, sin, cos, pi

class Vector3D:
    def __init__(self, x = 0, y = 0, z = 0, *, vector3D = None):
        if vector3D is None:
            self.x = x
            self.y = y
            self.z = z
        else:
            self.x = vector3D.x
            self.y = vector3D.y
            self.z = vector3D.z
            
    def isNull( self ) :
        return self.x == 0 and self.y == 0 and self.z == 0
        
    def set(self, x = 0, y = 0, z = 0, *, vector3D = None):
        if vector3D is None:
            self.x = x
            self.y = y
            self.z = z
        else:
            self.x = vector3D.x
            self.y = vector3D.y
            self.z = vector3D.z
        
        return self
    
    def __add__(self, other):
        vNew = Vector3D( vector3D = self )

        if isinstance(other, Vector3D) :
            vNew.x += other.x
            vNew.y += other.y
            vNew.z += other.z
        else:
            vNew.x += other[0]
            vNew.y += other[1]
            vNew.z += other[2]
            
        return vNew

    def __sub__(self, other):
        vNew = Vector3D( vector3D = self )

        if isinstance(other, Vector3D) :
            vNew.x -= other.x
            vNew.y -= other.y
            vNew.z -= other.z
        else:
            vNew.x -= other[0]
            vNew.y -= other[1]
            vNew.z -= other[2]

        return vNew
    
    def __mul__(self, other):
        if isinstance(other, Vector3D) :
            return self.x * other.x + self.y * other.y + self.z * other.z
        
        else :
            vNew = Vector3D( vector3D = self )

            vNew.x *= other
            vNew.y *= other
            vNew.z *= other

            return vNew
            
    def abs( self ):
        return sqrt(self.x * self.x + self.y * self.y + self.z * self.z)
            
    def normalize( self ):
        abs = self.abs()

        if abs != 0 :
            self.x /= abs
            self.y /= abs
            self.z /= abs
                
        return self

    def normalized( self ):
        return Vector3D( vector3D = self ).normalize()
                
    def cross( self, other ):
        return Vector3D(
            self.y*other.z - self.z*other.y,
            self.z*other.x - self.x*other.z,
            self.x*other.y - self.y*other.x
        )
        
    def __str__( self ):
        return str( "[{}, {}, {}]".format(self.x, self.y, self.z) )

    def rotated( self, alphaX = 0, alphaY = 0, alphaZ = 0, *, axis = None, angle = None ):
        if axis is None:
            cX = cos( alphaX )
            cY = cos( alphaY )
            cZ = cos( alphaZ )
            sX = sin( alphaX )
            sY = sin( alphaY )
            sZ = sin( alphaZ )
            
            vNew = Vector3D()
            
            vNew.x = (cY*cZ)*self.x + (sX*sY*cZ-cX*sZ)*self.y + (cX*sY*cZ+sX*sZ)*self.z
            vNew.y = (cY*sZ)*self.x + (sX*sY*sZ+cX*cZ)*self.y + (cX*sY*sZ-sX*cZ)*self.z
            vNew.z = (-sY  )*self.x + (sX*cY         )*self.y + (cX*cY         )*self.z
            
            return vNew
            
        else:
            vNew = axis.normalized() # reuse object!
            nx = vNew.x
            ny = vNew.y
            nz = vNew.z
            
            c = cos( angle )
            s = sin( angle )
            t = 1-c
            
            vNew.x = (t*nx*nx + c   )*self.x + (t*nx*ny - s*nz)*self.y + (t*nx*nz + s*ny)*self.z
            vNew.y = (t*nx*ny + s*nz)*self.x + (t*ny*ny + c   )*self.y + (t*ny*nz - s*nx)*self.z
            vNew.z = (t*nx*nz - s*ny)*self.x + (t*ny*nz + s*nx)*self.y + (t*nz*nz + c   )*self.z
            
            return vNew

In [12]:
########################################################################        
# MPU6050
########################################################################        
# g = 9.8070 : Bayern-Süd (Nieder- und Oberbayern, Schwaben)
# g = 9.8081 : Baden-Württemberg, Bayern-Nord (Franken, Oberpfalz)
# g = 9.8107 : Hessen, Nordrhein-Westfalen, Rheinland-Pfalz, Saarland, Sachsen, Thüringen
# g = 9.8130 : Berlin, Brandenburg, Bremen, Hamburg, Mecklenburg-Vorpommern, Niedersachsen, Sachsen-Anhalt, Schleswig-Holstein 
# Source: https://de.wikipedia.org/wiki/Gravitationszone

import time
import math # for pi and sqrt
from micropython import const

class MPU6050:
    # Data register and buffer
    REGID_PWR_SLEEP_1      = const(0x6B)  # Primary power/sleep control register
    REGID_SENSOR_DATA_BASE = const(0x3B)

    READ_BUFFER_SIZE = const(14)
    
    # Acceleration settings
    REGID_ACCELERATROR_CONFIG = const(0x1C)
    
    ACCELERATOR_RANGE_2_G  = const(0)  # +/-  2 g (default)
    ACCELERATOR_RANGE_4_G  = const(1)  # +/-  4 g
    ACCELERATOR_RANGE_8_G  = const(2)  # +/-  8 g
    ACCELERATOR_RANGE_16_G = const(3)  # +/- 16 g

    ACCELERATOR_SENS_SCALE_2G = 16384 # = 2^14
    GRAVITY_DEFAULT = 9.8070 # south of bavaria

    
    # Gyroscope settings
    REGID_GYROSCOPE_CONFIG = const(0x1B)
    
    GYROSCOPE_RANGE_250_DPS  = const(0)  # +/-  250 deg/s (default)
    GYROSCOPE_RANGE_500_DPS  = const(1)  # +/-  500 deg/s
    GYROSCOPE_RANGE_1000_DPS = const(2)  # +/- 1000 deg/s
    GYROSCOPE_RANGE_2000_DPS = const(3)  # +/- 2000 deg/s

    GYROSCOPE_SENS_SCALE = const(131) # math.pi/(180*131)

    
    def __init__(self, i2c, addr=0x68):
        self.iic  = i2c
        self.addr = addr

        self.iic.writeto_mem(self.addr, MPU6050.REGID_PWR_SLEEP_1, bytearray([0x80])) # Reset MPU6050 (b'\x80' not enabled in Esp8266-MicroPython: 2022-02-02)
        time.sleep( 0.1 )
        self.iic.writeto_mem(self.addr, MPU6050.REGID_PWR_SLEEP_1, bytearray([0x00])) # Disable sleep
        
        self.aDataByte = bytearray(MPU6050.READ_BUFFER_SIZE)
        self.aDataInt  = [0]   * int(MPU6050.READ_BUFFER_SIZE/2)
        self.aData     = [0.0] * int(MPU6050.READ_BUFFER_SIZE/2)
        self.aDataNorm = [0.0] * int(MPU6050.READ_BUFFER_SIZE/2)
        
        self.acceleratorSensScale = MPU6050.GRAVITY_DEFAULT/MPU6050.ACCELERATOR_SENS_SCALE_2G
        self.gyroscopeSensScale   = math.pi/(180*MPU6050.GYROSCOPE_SENS_SCALE)

        self.acceleratorNormScale = 1.0
        
    def setAcceleratorRange( self, accelRange ):
        self.iic.writeto_mem(self.addr, MPU6050.REGID_ACCELERATROR_CONFIG, bytearray([accelRange<<3]))
        # sensitifity scale: 16384, 8192, 4096, 2048 (see manual)
        self.acceleratorSensScale = MPU6050.GRAVITY_DEFAULT*(1<<accelRange)/MPU6050.ACCELERATOR_SENS_SCALE_2G

    def setGyroscopeRange( self, gyroRange ):
        self.iic.writeto_mem(self.addr, MPU6050.REGID_GYROSCOPE_CONFIG, bytearray([gyroRange<<3]))
        # sensitifity scale: 131, 65.5, 32.8, 16.4 (see manual)
        self.gyroscopeSensScale = math.pi/(180*round(MPU6050.GYROSCOPE_SENS_SCALE/(1<<gyroRange),1))
                
    @staticmethod
    def bytesToInt(byteHigh, byteLow):
        if not byteHigh & 0x80:
            return byteHigh << 8 | byteLow
        else:
            return - ( ((byteHigh^0xff) << 8) | (byteLow^0xff) + 1 ) # negativ values are in two's complement, so return  -(two's complement)
    
    def getRawDataIntArray(self):
        self.iic.readfrom_mem_into(self.addr, MPU6050.REGID_SENSOR_DATA_BASE, self.aDataByte ) # 0x3B: base address for sensor data reads

        for iIndex in range( 0, MPU6050.READ_BUFFER_SIZE, 2 ):
            self.aDataInt[ iIndex>>1 ] = MPU6050.bytesToInt( self.aDataByte[ iIndex ], self.aDataByte[ iIndex +1] )

        return self.aDataInt
    
    
    # 0, 1, 2: Accelerator: x, y, z: in m/s^2
    # 3      : Temperature: value ( convert to °C: value / 340.00 + 36.53 )
    # 4, 5, 6: Gyroscope  : x, y, z: in 1/s (omega)

    def getDataArray( self, normalize = False ):
        self.getRawDataIntArray()
        
        self.aData[0] = self.aDataInt[0] * self.acceleratorSensScale
        self.aData[1] = self.aDataInt[1] * self.acceleratorSensScale
        self.aData[2] = self.aDataInt[2] * self.acceleratorSensScale

        self.aData[3] = self.aDataInt[3]/340 + 36.53
        
        self.aData[4] = self.aDataInt[4] * self.gyroscopeSensScale
        self.aData[5] = self.aDataInt[5] * self.gyroscopeSensScale
        self.aData[6] = self.aDataInt[6] * self.gyroscopeSensScale

        if normalize == True :
            for iIndex in range( 4, 7 ):
                self.aData[iIndex] -= self.aDataNorm[iIndex]
        
            for iIndex in range( 3 ):
                self.aData[iIndex] *= self.acceleratorNormScale
                
        return self.aData   

        
    def normalize( self, iSamples = 100 ):
        for iCount in range( 100 ): # first samples are sometimes [0,0,0,0,0,0,0] : skip.
            self.getRawDataIntArray()
            
            if self.aDataInt[0] != 0 or self.aDataInt[1] != 0 or self.aDataInt[2] != 0:
                self.aDataNorm = self.getDataArray()[:]
                break
                
        # now calcualte average from the next iSamples samples:
        for iN in range( 2, iSamples+1 ):
            self.getDataArray()
            for iIndex in range(int(MPU6050.READ_BUFFER_SIZE/2)):
                self.aDataNorm[iIndex] = (self.aDataNorm[iIndex]*(iN-1) + self.aData[iIndex])/iN
          
        self.acceleratorNormScale = MPU6050.GRAVITY_DEFAULT / math.sqrt(self.aDataNorm[0]**2 + self.aDataNorm[1]**2 + self.aDataNorm[2]**2)
        self.aDataNorm[3] = 0 # Temperature needs no normalisation.              
    

In [13]:
########################################################################        
# AverageFilter
########################################################################        

class AverageFilter :
    def __init__(self,len):
        self.length   = 0
        self.iLast    = len
        self.aData    = [0]*len # [i for i in range( len )]
        self.fAverage = 0
        self.isFull   = False
        
    def get(self, iIndex):
        if self.isFull == True :
            return self.aData[ (self.iLast + iIndex) % self.length ]
        else :
            return self.aData[ self.iLast + (iIndex % self.length) ]

    def getAverage(self):
        return self.fAverage
        
    def add(self,val):
        if self.isFull == True :
            self.iLast = (self.iLast - 1) % self.length
            self.fAverage += ((val - self.aData[self.iLast])/self.length)
            self.aData[self.iLast] = val
        else :
            self.iLast-=1
            self.aData[self.iLast] = val
            self.fAverage *= self.length
            self.length +=1
            self.fAverage = (self.fAverage + val)/self.length
            if len(self.aData) == self.length :
                self.isFull = True

    def print(self) :
        if self.isFull == True :
            print( self.__class__, self.aData[self.iLast:] + self.aData[:self.iLast] )
        else :
            print( self.__class__, self.aData[self.iLast:] )
            
    def __str__(self) :
        return( "{}  {} ==> Average = {}".format( self.__class__, self.aData[self.iLast:] + self.aData[:self.iLast] if self.isFull == True else self.aData[self.iLast:] , self.fAverage ) )

## Programm

In [14]:
#=====================================
# Constants

AVERAGE_SIZE   =  5

VIEW_SCALE     = 20
VIEW_SWAP_XY   = True
VIEW_REFLECT_X = False
VIEW_REFLECT_Y = True


#=====================================
# I2C for all types of used controllers

from machine import I2C, Pin
import info

if   info.TYPE == "Esp8266 Croduino Nova": i2c = I2C(   scl=Pin( 4), sda=Pin( 5))
elif info.TYPE == "Esp32 NodeMCU"        : i2c = I2C(1, scl=Pin(21), sda=Pin(22))
elif info.TYPE == "Esp32 HelTec"         : i2c = I2C(1, scl=Pin(15), sda=Pin( 4))
else : print( "FATAL ERROR: Unknown controller!" )



#=====================================
# class View
# responsible for showing the cube

from math import acos
import display

class View :
    display = display.Display(i2c)
    
    def __init__(self, i2c, *, scale = 20, swapXY = False, reflectX = False, reflectY = False ):
        self.swapXY   = swapXY
        self.scaleX   = -scale if reflectX else scale
        self.scaleY   = -scale if reflectY else scale
        
        self.vecAxis  = Vector3D()

        self.display.setCenter( 63, 31 )

    def line( self, v1, v2, dX = 0, dY = 0, color = 1 ):
        if self.swapXY :
            self.display.line( int(v1.y*self.scaleY) + dX, int(v1.x*self.scaleX) + dY, int(v2.y*self.scaleY) + dX, int(v2.x*self.scaleX) + dY, color )
        else :
            self.display.line( int(v1.x*self.scaleX) + dX, int(v1.y*self.scaleY) + dY, int(v2.x*self.scaleX) + dX, int(v2.y*self.scaleY) + dY, color )
        
    def draw( self, aVector, vecG ):
        
        self.display.clear()

        aVrot = [ vector.rotated( axis  = self.vecAxis.set( vecG.y, -vecG.x, 0 ),
                                  angle = acos( -vecG.z/vecG.abs() ) ) for vector in aVector ]

        for iIndex in range( 0, 4 ):
            self.line( aVrot[ iIndex  ], aVrot[  iIndex+4     ] )
            self.line( aVrot[ iIndex+4], aVrot[ (iIndex+1)%4+4] )

        self.line( aVrot[ 4 ], aVrot[ 6 ] )
        self.line( aVrot[ 5 ], aVrot[ 7 ] )

#        for iIndex in range( 0, 4 ):
#            self.line( aVrot[ iIndex+ 8], aVrot[ (iIndex+1)%4+ 8] )
#            self.line( aVrot[ iIndex+12], aVrot[ (iIndex+1)%4+12] )

        for iIndex in range( 0, 4 ):
            self.line( aVrot[ iIndex ], aVrot[ (iIndex+1)%4 ], dX =  1 )
            self.line( aVrot[ iIndex ], aVrot[ (iIndex+1)%4 ], dX = -1 )
            self.line( aVrot[ iIndex ], aVrot[ (iIndex+1)%4 ], dY =  1 )
            self.line( aVrot[ iIndex ], aVrot[ (iIndex+1)%4 ], dY = -1 )
            self.line( aVrot[ iIndex ], aVrot[ (iIndex+1)%4 ] )

        self.display.show()


# View instance:        
view = View( i2c, scale = VIEW_SCALE, swapXY = VIEW_SWAP_XY, reflectX = VIEW_REFLECT_X, reflectY = VIEW_REFLECT_Y )


#=====================================
# cube model

aVector = (
    Vector3D( 1, 1,  1 ), Vector3D( 1, -1,  1 ), Vector3D( -1, -1,  1 ), Vector3D( -1, 1,  1 ),
    Vector3D( 1, 1, -1 ), Vector3D( 1, -1, -1 ), Vector3D( -1, -1, -1 ), Vector3D( -1, 1, -1 )
)


#=====================================
# Accelerator

mpu6050 = MPU6050(i2c)
vecG    = Vector3D() # points in direction of earth gravity

# Reduce noise of accelerator:
aX = AverageFilter(AVERAGE_SIZE)
aY = AverageFilter(AVERAGE_SIZE)
aZ = AverageFilter(AVERAGE_SIZE)


#=====================================
# loop "forever"

try:
    while True:
        aData = mpu6050.getDataArray()
        
        aX.add( aData[0] )
        aY.add( aData[1] )
        aZ.add( aData[2] )
        
        vecG.set( aX.getAverage(), aY.getAverage(), aZ.getAverage() )

        if not vecG.isNull(): # if 0-vector, something went wrong
            view.draw( aVector, vecG )
        
except KeyboardInterrupt:
    print( "User break." )

.....................................................................................................................................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] device reports readiness to read but returned no data (device disconnected or multiple access on port?)

